<a href="https://colab.research.google.com/github/sametgumus212/Flask/blob/main/step_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash
!pip install dash-bootstrap-components
!pip install pyreadstat
!pip install spss-converter
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.9 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
import spss_converter
from spss_converter import read, write
import pandas as pd
import pyreadstat
import json

path='/content/drive/MyDrive/sav/ebs73.1_2010.sav'
# path='/content/drive/MyDrive/sav/ebs63.1_2005.sav'
# ebs38.1_1992.sav
# ebs55.2_2001.sav
#ebs63.1_2005.sav
#ebs73.1_2010.sav
# ebs79.2_2013.sav
# ebs95.2_2021.sav

df, metadata = spss_converter.to_dataframe(path)

metadata_dict =metadata.to_dict()#convert to dictionary to put it in the DB

column_metadata = metadata_dict['column_metadata']

#converting  column metadata to dict
#column_metadata['v6'].to_dict()
dictio = {}
for key, value in column_metadata.items():
    dictio.update({key: value.to_dict()})

mt = pd.DataFrame(dictio).transpose()

mt['value_metadata'] = mt['value_metadata'].apply(json.dumps)#value labels
mt['missing_range_metadata'] = mt['missing_range_metadata'].astype(str)#user missing
mt['label'] = mt['label'].apply(lambda x: mt.name[mt['label'].isna()].values[0] if pd.isna(x) else x)#variable labels
mt = mt.drop(columns=['name', 'alignment', 'display_width', 'storage_width'])


import pandas as pd
import sqlite3
conn = sqlite3.connect('database.db')
# df.to_sql('df2005', conn)
# mt.to_sql('meta2005', conn)
df.to_sql('df2010', conn)
mt.to_sql('meta2010', conn)
conn.close()

In [27]:
###AHMET step one complete

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_table
from dash.dependencies import Input, Output, State
import sqlite3
import pandas as pd

#app = dash.Dash(_name_)
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])# CYBORG COSMO select themes from: https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/

######################  start of Değişken tanımlanmaları ###################################
txt='''
### 1.1.1. List of available surveys

First select the survey you are interested in by checking the box. After finishing  the preprocessing and preparing the data, you can download it to yıoıur computer. If you want to analyze multiple surveys together as mentioned in the explanation [link], prepare each data at a time and download to your computer. After complete, you can merge them for analysis.
Below is a list of surveys. You can get the {codebook} by clicking on the codebook button.

1.   EB S&T surveys

Eurobarometer 79.2    April-May 2013     ZA No. 5688 {https://search.gesis.org/research_data/ZA5688?doi=10.4232/1.12577}

to view the codebook:

[EB 2013 meta] submit button

After vewing the codebook, you can decide on which variables to work with
select the variables
submit button
[drop box ], multi


..........

Eurobarometer 63.1 (Jan-Feb 2005) {https://search.gesis.org/research_data/ZA4233?doi=10.4232/1.10965}

[EB 2005] codebook

......................

Eurobarometer 31     March-April 1989     ZA No.  1750 {https://search.gesis.org/research_data/ZA1750?doi=10.4232/1.10888}
'''


# Define empty global variables
mt1 = pd.DataFrame()
data_df = pd.DataFrame()
metadata_df = pd.DataFrame()
#selected_vars_div = html.Div(id='selected-vars')#unghost if selected var updated
#tool tip dictionary for column name definitions
ttip={
        'index': 'variable name',
        'label': 'variable label',
        'measure': 'level of measurement',
        'value_metadata' : 'value label',
        'missing_range_metadata' :'miss',
        'missing_value_metadata' : 'missing',
    }


text_table='''
When adding Dash HTML Components, it works  by appending items to the
children parameter of the main html.Div element. In this case, every added item
occupies the full width of the screen and takes as much screen height as it needs to display
its contents. The order of the elements in the list determines their order of display on the
screen as well.,

we can change this With Bootstrap, you simply define a column, and that
in turn behaves as an independent screen, displaying its elements on top of one another,
with each element occupying the full width of this mini screen. Columns' widths can
also be specified in powerful and flexible ways. The grid system divides the screen into
12 columns, and the width of a column can be specified by using a number from 1 to 12
inclusive. check Dabbas book, pages 15-16. It shows how the columns can be defined, and how they would change
for screens of different size:

'''
#===================================== end of Değişken tanımlanmaları ===================================

#////////////////////////////////////////////////////////////////////////////////////////////////////////

#########################################################################################################
# Define the app layout
#############################            LAYOUT                        ##################################
### user input to fetch meta from database, --enrich w html, color etc...
#########################################################################################################
selected_vars_div = html.Div(id='selected-vars') # seçilen variablelerin gösterimi
selected_vars_div_two = html.Div(id='selected-vars_two')# 2. sayfada seçilen variablelerin seçimi
app.layout = html.Div([
    html.H1('Fetch the surveys',
          style={'color': 'blue',
          'fontSize': '40px'}),
    html.H2('Select one from the following list'),
    dbc.Tabs([####tabs begining
        dbc.Tab([# start of first tab   *****************************************************************

                 dcc.Markdown(txt),   #sayfanın üst yazısının gösterimi (markdawn 1)

    html.Div([###########User input format
            html.Br(),#break
              dbc.Row([#in one row two submit buttons in two columns
                 dbc.Col([###First submit column
                  dbc.Label('Enter the name of the metadata'),
                  html.Br(),
                  dcc.Input(
                    id='table-input',
                    type='text',
                    placeholder='i.e. meta2005, meta2010...',
                    value='meta2005'
        ),
        html.Button('Search', id='submit-btn', n_clicks=0), #
        ####meta-table output
        html.Div(id='output-div'),
        ], md=12, lg=5), # ekrandaki kaplayacağı alan
                       ##----------1. kolonun sonu
                 dbc.Col([
                         ###########submit button for selecetd rows

                    dbc.Label('After selecting, submit.'),
                    html.Br(),
                    selected_vars_div, # do tjis for updating variables as checked
                    html.Button('Submit clicked', id='submit-button', n_clicks=0),
                    html.Div(id='selected-rows'),
                 ], md=12, lg=5),

        ]),
    ]),



        dbc.Row([
            # Start of first column
            dbc.Col([
    ########################  DATATABLE output format
    dash_table.DataTable(
        id='datatable',
        ##########needed for interactive datatable  https://dash.plotly.com/datatable/interactivity, which is needed for dt wirh checklist
        columns=[],#creates a list of dictionaries, where each dictionary has a name and id key-value pair. These key-value pairs are generated from the column names in the mt1
        data=[],# converts the mt1 object into a list of dictionaries using the .to_dict('records') method. Each dictionary represents a row in the mt1
        ################################################################# the empty lists will be updated later in the code
        ###rows per page
        page_current=0,
        page_size=25,
        tooltip_header=ttip,  # tooltip dictionary i burda tooltip olarak alıyor

        ###for different colors between rows
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        ######title of the datatable
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'textDecoration': 'underline',
            'textDecorationStyle': 'dotted',
        },
        #this is for selecting the row with checkboxes, https://dash.plotly.com/datatable/interactivity
        editable=False,
        row_selectable='multi',
        selected_rows=[],
        ### this sets the style of the datatable
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        style_cell={'textAlign': 'left',
        'textOverflow': 'ellipsis'},#alignment in cells,
         tooltip_delay=0,
         tooltip_duration=None
       ),
                ], md=7),#end of first column, dt

#1. sayfadaki datatablenin yanında çıkacak yazı
                    dbc.Col([##### column for the text on the right hand side of dt
                dcc.Markdown(id='markdown-content', children=text_table, style={'display': 'none'})
                  ],  md=5),#end of column
        ]),

# 1. tabın labılı
          ], label='fetch the data'), #tab label,end  of first tab   **********   1. sayfanın sonu *****************
# 1. tabın sonu --------??? acaba ayrıştırma yavaşlatıyormu



#####################################################################################

######################################### -----SAYFA 2...........####################

#####################################################################################
# 2. tabın başlangıcı
        dbc.Tab([
          html.Ul([
            html.Br(),
            html.Li('Preprocess the data, lorem ipsum '),
            ]),
          ###########**************** show result start
             html.Br(),
#============******* çalışma alanı radio button**********================

#buraya radio button eklenecek.
# textbox manuel girdi için
#===============******* yapılacak end of çalışma alanı radio button **********=============

          # sayfa 2 deki çıktıları gösteren button (çıktı = metadata_df vs))
      html.Div([

        html.Button('show results', id='results-btn', n_clicks=0)
    ]),
          html.Br(),
#=======================*********yapılacak çalışma alanı****************===============
# dict için button vs

#=======================***************end of çalışma alanı**********===============
          ################################**********************************
    html.Br(),
                 #çıktıların 2. sayfada gösterilmesi için  metadata_df
  dash_table.DataTable(id="metadf_out",columns=[],#creates a list of dictionaries, where each dictionary has a name and id key-value pair. These key-value pairs are generated from the column names in the mt1
        data=[],# converts the mt1 object into a list of dictionaries using the .to_dict('records') method. Each dictionary represents a row in the mt1
        ################################################################# the empty lists will be updated later in the code
        ###rows per page
        page_current=0,
        page_size=25,
        tooltip_header=ttip,  # tooltip dictionary i burda tooltip olarak alıyor

        ###for different colors between rows
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        ######title of the datatable
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'textDecoration': 'underline',
            'textDecorationStyle': 'dotted',
        },
        #this is for selecting the row with checkboxes, https://dash.plotly.com/datatable/interactivity
        editable=False,
        row_selectable='multi',
        # selected_rows=[],
        ### this sets the style of the datatable
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        style_cell={'textAlign': 'left',
        'textOverflow': 'ellipsis'},#alignment in cells,
         tooltip_delay=0,
         tooltip_duration=None
    ),


    html.Br(),
          #çıktıların 2. sayfada gösterilmesi için  data_df
          #Not gereksiz görülüp kaldırılabilinir aynı zamanda (hocamın fikrini al)
  dash_table.DataTable(id="datadf_out",columns=[],#creates a list of dictionaries, where each dictionary has a name and id key-value pair. These key-value pairs are generated from the column names in the mt1
        data=[],# converts the mt1 object into a list of dictionaries using the .to_dict('records') method. Each dictionary represents a row in the mt
        page_current=0,
        page_size=25,
        tooltip_header=ttip,  # tooltip dictionary i burda tooltip olarak alıyor

        ###for different colors between rows
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        ######title of the datatable
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'textDecoration': 'underline',
            'textDecorationStyle': 'dotted',
        },
        #this is for selecting the row with checkboxes, https://dash.plotly.com/datatable/interactivity
        editable=False,
        row_selectable='multi',
        # selected_rows=[],
        ### this sets the style of the datatable
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        style_cell={'textAlign': 'left',
        'textOverflow': 'ellipsis'},#alignment in cells,
         tooltip_delay=0,
         tooltip_duration=None
    ),
 html.Br(),
          ##########************* end of show result

          ],    label='Prepocess'),  # 2. tabın sonu ---------

    ]), # tüm tabın kapanışı


],

style={'backgroundColor': '#E5ECF6'} #style of all layout, this tim ethe color
)
################################################################################

##################                callback starttttt          #################

#############################################################3##################


# Define the callback function to fetch the input table from the database
@app.callback(
    Output('markdown-content', 'style'), # markdownda gözükecek yazı
    Output('output-div', 'children'),  # output i.e mt1
    [Input('submit-btn', 'n_clicks')],  # submit
    [State('table-input', 'value')]  # user entry
)
def fetch_table(n_clicks, table_name):
    global mt1 # Use the global variable

    if n_clicks > 0:


        # Connect to the database
        conn = sqlite3.connect('database.db')

        try:
            mt1 = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)
            # visible = 'block'# if n_clicks >0 else 'none'
            return {'display': 'block'},html.Div()
        except Exception as e:
            # visible='none'
            # {'display': 'none'}
            return {'display': 'none'},html.Div(f'Error: {e}')

        conn.close()

    return {'display': 'none'},html.Div()
# Connect to the database and retrieve data
def fetch_data(selected_columns):
    global data_df
    global metadata_df
    global met1
    global met
    conn = sqlite3.connect('database.db')
    selected_table_df="df{}".format(selected_table[-4:])

    # Select desired columns from ebs631_05 table
    data_df = pd.read_sql_query(f"SELECT {', '.join(selected_columns)} FROM {selected_table_df}", conn) #df2005

    # Filter metadata_df based on selected variables using meta2005 table
    metadata_df = pd.read_sql_query(f"SELECT * FROM {selected_table}", conn) #meta2005
    metadata_df = metadata_df[metadata_df['index'].isin(selected_columns)]

    conn.close()
    return metadata_df

# Define the callback function to display the datatable and selected rows
@app.callback(
    Output('datatable', 'columns'),
    Output('datatable', 'data'),
    [Input('submit-btn', 'n_clicks')],
    [State('table-input', 'value')]
)
def display_datatable(n_clicks, table_name):
    global met1  # Use the global variable

    if n_clicks > 0:
        # Connect to the database

        conn = sqlite3.connect('database.db')

        try:


            met1 = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)
            columns = [{'name': i, 'id': i} for i in met1.columns]
            data = met1.to_dict('records')
        except Exception as e:
            return [], []

        # Close the connection
        conn.close()

        return columns, data

    else:
        return [], []


@app.callback(
    Output('selected-vars', 'children'),
    [Input('datatable', 'selected_rows')]

)
def update_selected_vars(selected_rows):
    global met1
    global met
    selected_columns = [met1.iloc[i]['index'] for i in selected_rows]   #db
    return html.Div(f'Selected Variables: {", ".join(selected_columns)}')

@app.callback(
    Output('selected-rows', 'children'),
    [Input('submit-button', 'n_clicks')],
    [State('datatable', 'selected_rows')]
)
def update_selected_rows(n_clicks, selected_rows):
    global data_df
    global metadata_df
    global met1


    if n_clicks > 0:
        selected_columns = [met1.iloc[i]['index'] for i in selected_rows]  #db
        fetch_data(selected_columns)

        return html.Div([
            html.Div(f'Total number of variables: {len(selected_rows)}'),
            html.Div(f'Selected Variables: {", ".join(selected_columns)}'),
        ])

    else:
        return 'Total number of variables:'

#================================================

# second page
############################## show results ###########################################
@app.callback(
    Output('metadf_out', 'columns'),

    Output('metadf_out', 'data'),
    Output('datadf_out','data'),
     Output('datadf_out','columns'),
    [Input('results-btn', 'n_clicks')],
    # [State('datatable', 'selected_rows')]
)

def display_result(n_clicks):
  global data_df
  global metadata_df
  if n_clicks>0:
    dft=data_df.to_dict('records')
    columns_dft = [{'name': i, 'id': i} for i in data_df.columns]
    mft=metadata_df.to_dict('records')
    columns_mft = [{'name': i, 'id': i} for i in metadata_df.columns]
    return columns_mft,mft,dft,columns_dft
  else:
    return [],[],[],[]

#################################################################



    # Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# 1-selected row reset
# 2- prepocess kısmında çıktıların gözükmesi (meta_df,,data_df)
# 3- html unordered list kısmında markdown olayına dönüştür .....
# 4- seçilen source kısmının ilgili meta name textbox kısmında çıkcak.
# markdownu yerleştir.

<IPython.core.display.Javascript object>